<a href="https://colab.research.google.com/github/Arthur-Barreto/Machine-Vision/blob/main/VisComp_Class_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class 4: Handling Noise

## Preliminaries

Run the cell below to download the class pack.

In [ ]:
import gdown

gdown.download(id='1VP6k_Fd6hNlnJ0f2bvD_VvOAIudrU2z_')

!unzip -o '04.zip'
!rm '04.zip'

Run the cell below to import the class modules.

If you get import warnings, try using **Ctrl+m .** (notice there is a dot there) to restart the kernel.

In [ ]:
import numpy as np

from sdx import *

## Loading a specific image

The idea of this notebook is running the same code for different images.

By changing the two parameters below, you have 6x3=18 images to choose.

Possible values of `NAME`: `atletica`, `consulting`, `smash`, `insper`, `harvard`, `informatica`

In [ ]:
NAME = 'atletica'

Possible values of `LEVEL`: `8`, `16`, `32`

In [ ]:
LEVEL = 8

The cell below loads the chosen image, converts it to gray, and converts it to `float`.

Play a bit with the two parameters to see the differences between the images.

In [ ]:
image = cv_grayread(f'{NAME}-{LEVEL}.png')

cv_imshow(image)

## Part 1: average blur

For each pair of coordinates `(y, x)`, consider its neighborhood of size `n`, that is, the region of interest from rows `y-n//2` to `y+n//2` and columns `x-n//2` + `x+n//2` (inclusive). We will always consider that `n` is odd, so that `(y, x)` is the exact center of this region.

### Challenge (easy level):

Write a function that, given an image, creates a new image where each pixel is the *average of the neighborhood of size `3`* of the original pixel.

You can use four loops, like in Class 1, but you are free to use NumPy resources to reduce the number of loops.

In [ ]:
def average_blur(image):
    final_image = image.copy()
    height, widht = final_image.shape
    n = 3

    for i in range(n//2, widht-n//2):
        for j in range(n//2, height-n//2):
            roi = image[i-n//2 : i+n//2, j-n//2:j+n//2]
            final_image[i,j] = np.mean(roi)

    return final_image

In [ ]:
cv_imshow(average_blur(image))

### Challenge (medium level):

Generalize your function for an arbitrary given size `n`.

In [ ]:
def average_blur(image, n):
    final_image = image.copy()
    height, widht = final_image.shape

    for i in range(n//2, widht-n//2):
        for j in range(n//2, height-n//2):
            roi = image[i-n//2 : i+n//2, j-n//2:j+n//2]
            final_image[i,j] = np.mean(roi)

    return final_image

In [ ]:
cv_imshow(average_blur(image, 31))

### Challenge (hard level)

Using only NumPy, rewrite the previous function without any loops.

In [ ]:
from scipy.signal import convolve2d

def average_blur(image, n):
    data = image.copy()
    # Define blurring kernel
    blur_kernel = np.ones((n, n)) / (n ** 2)

    # Pad the data to handle borders
    padded_data = np.pad(data, n // 2, mode='wrap')

    # Perform the blurring operation using numpy's strides
    blurred_data = np.lib.stride_tricks.sliding_window_view(padded_data, (n, n))
    blurred_data = np.sum(blurred_data * blur_kernel, axis=(2, 3))

    return blurred_data

In [ ]:
cv_imshow(average_blur(image, 31))

## Part 2: gaussian blur

### Challenge (easy level)

Write a function that, given an image, creates a new image where each pixel is the *sum of the neighborhood of size `3`* of the original pixel, but weighted by the gaussian function below. You *can* assume that $\sigma = 1$ and you **must** assume that the origin `(0, 0)` in the function is the center of the neighborhood.

$$\frac{e^{-(x^2+y^2)/2\sigma^2}}{2\pi\sigma^2}$$

You can use four loops, like in Class 1, but you are free to use NumPy resources to reduce the number of loops.

In [ ]:
def gaussian_blur(image):
    n = 3
    sigma = 1
    img = image.copy()
    kernel = np.fromfunction(
        lambda x, y: (1/ (2 * np.pi * sigma**2)) * np.exp(-((x - n//2)**2 + (y - n//2)**2) / (2 * sigma**2)),
        (n, n)
    )
    kernel = kernel / np.sum(kernel)

    blurred_image = np.zeros_like(img, dtype=float)
    padded_image = np.pad(img, n//2, mode='wrap')

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            blurred_image[i, j] = np.sum(kernel * padded_image[i:i+n, j:j+n])

    return blurred_image

In [ ]:
cv_imshow(gaussian_blur(image))

### Challenge (medium level):

Generalize your function for an arbitrary given size `n` and an arbitrary `sigma`.

In [ ]:
def gaussian_blur(image, n, s):
    img = image.copy()
    kernel = np.fromfunction(
        lambda x, y: (1/ (2 * np.pi * s**2)) * np.exp(-((x - n//2)**2 + (y - n//2)**2) / (2 * s**2)),
        (n, n)
    )
    kernel = kernel / np.sum(kernel)

    blurred_image = np.zeros_like(img, dtype=float)
    padded_image = np.pad(img, n//2, mode='wrap')

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            blurred_image[i, j] = np.sum(kernel * padded_image[i:i+n, j:j+n])

    return blurred_image

In [ ]:
cv_imshow(gaussian_blur(image, 3, 1))

### Challenge (hard level)

Using only NumPy (okay, maybe SciPy too), rewrite the previous function without any loops.

In [ ]:
from scipy.signal import convolve2d

def gaussian_blur(image, n, s):
    img = image.copy()
    kernel = np.fromfunction(
        lambda x, y: (1/ (2 * np.pi * s**2)) * np.exp(-((x - n//2)**2 + (y - n//2)**2) / (2 * s**2)),
        (n, n)
    )
    kernel = kernel / np.sum(kernel)

    blurred_image = convolve2d(img.astype(float), kernel, mode='same', boundary='wrap')

    return blurred_image

In [ ]:
cv_imshow(gaussian_blur(image, 3, 1))

You can click on the toc.png tab to the left to browse by section.